# Exercise 16: Build, train and evaluate a neural network.

Build, train and evaluate a NN based on the following instructions:
- The training dataset has **32 features**;
- The task is **binary classification**;
- Use the **SGD** optimizer;
- Use the **BinaryCrossEntropy** loss;
- Use the **accuracy** metric.
- The model should contain:
    - **Dense layer** 1
    - **ReLU activation** layer 1
    - **Dense layer** 2
    - **ReLU activation** layer 2
    - Output **Dense layer**
    - **Sigmoid activation** layer
- The dense layers should **reduce the number of units to half (except the last one)**;
- Train the NN for **100 epochs**, with **batch size of 16** with a **learning rate of 0.01**;
- Test the model with **k fold cross validation** (hint: use the functions implemented in class 8).

In [1]:
import numpy as np

from si.data.dataset import Dataset
from si.neural_networks.neural_network import NeuralNetwork
from si.neural_networks.optimizers import SGD
from si.neural_networks.losses import BinaryCrossEntropy
from si.metrics.accuracy import accuracy

from si.neural_networks.layers import DenseLayer
from si.neural_networks.activation import ReLUActivation, SigmoidActivation

from si.model_selection.cross_validate import k_fold_cross_validation

# Setting a global random seed for reproducible results
np.random.seed(42)

# Step 1: Simulating a training dataset
# Dataset with 32 features and binary classification labels (0, 1)
samples_size = 500                                  # Set number of samples for our dataset

X = np.random.rand(samples_size, 32)                # sample_size, 32 features
y = np.random.randint(0, 2, size=(samples_size, 1)) # Setting binary labels

dataset = Dataset(X=X, y=y)
print(dataset)

In [2]:
# Visualize the dataset as a pandas.DataFrame
df = dataset.to_dataframe()
df.head()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,...,0.366362,0.456070,0.785176,0.199674,0.514234,0.592415,0.046450,0.607545,0.170524,0
1,0.065052,0.948886,0.965632,0.808397,0.304614,0.097672,0.684233,0.440152,0.122038,0.495177,...,0.921874,0.088493,0.195983,0.045227,0.325330,0.388677,0.271349,0.828738,0.356753,0
2,0.280935,0.542696,0.140924,0.802197,0.074551,0.986887,0.772245,0.198716,0.005522,0.815461,...,0.637557,0.887213,0.472215,0.119594,0.713245,0.760785,0.561277,0.770967,0.493796,0
3,0.522733,0.427541,0.025419,0.107891,0.031429,0.636410,0.314356,0.508571,0.907566,0.249292,...,0.539342,0.807440,0.896091,0.318003,0.110052,0.227935,0.427108,0.818015,0.860731,1
4,0.006952,0.510747,0.417411,0.222108,0.119865,0.337615,0.942910,0.323203,0.518791,0.703019,...,0.239562,0.144895,0.489453,0.985650,0.242055,0.672136,0.761620,0.237638,0.728216,0


In [3]:
# Dataset characteristics
df.describe()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,y
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.496848,0.491304,0.472878,0.464821,0.504960,0.517980,0.514389,0.497113,0.504867,0.471078,...,0.497239,0.489826,0.502059,0.512914,0.492155,0.476725,0.512097,0.492978,0.494914,0.518000
std,0.284548,0.292714,0.288619,0.282959,0.296135,0.286709,0.293445,0.285115,0.283197,0.292372,...,0.290264,0.289000,0.286316,0.282233,0.294545,0.283982,0.281530,0.289454,0.296326,0.500176
min,0.001873,0.001112,0.000310,0.000053,0.006047,0.001320,0.000554,0.008223,0.000598,0.004360,...,0.000031,0.005324,0.004482,0.000410,0.004187,0.002141,0.000243,0.000906,0.000241,0.000000
25%,0.257543,0.233125,0.215986,0.226093,0.242166,0.275834,0.252760,0.265514,0.265846,0.215167,...,0.251051,0.238583,0.253425,0.271758,0.246486,0.241321,0.277226,0.246743,0.226610,0.000000
50%,0.503568,0.504965,0.471856,0.441295,0.494128,0.521523,0.533532,0.492804,0.527330,0.455616,...,0.488272,0.489994,0.489954,0.506643,0.485161,0.453506,0.506351,0.476470,0.484111,1.000000
75%,0.742750,0.735912,0.703383,0.692961,0.764477,0.769395,0.765696,0.753219,0.725879,0.720878,...,0.751326,0.739817,0.751629,0.755392,0.755583,0.718424,0.761707,0.755240,0.753410,1.000000
max,0.997749,0.999558,0.999925,0.996763,0.996991,0.999673,0.999207,0.999505,0.996145,0.999459,...,0.995802,0.999414,0.997885,0.999805,0.996734,0.999350,0.998561,0.997934,0.998905,1.000000


In [4]:
# Checking if dataset has null values (just in case)
print(f"Dataset has NULL values?: {df.isnull().values.any()}")

Dataset has NULL values?: False


In [5]:
# Step 2: Configure the Neural Network
# Initialize the neural network with the defined hyperparameters
nn = NeuralNetwork(
    epochs=100,                 # Trained for 100 epochs
    batch_size=16,              # Trained with a batch size of 16
    optimizer=SGD,              # Use the SGD optimizer
    learning_rate=0.01,         # Trained with a learning rate of 0.01
    verbose=True,
    loss=BinaryCrossEntropy,    # Use the BinaryCrossEntropy loss
    metric=accuracy             # Use the accuracy metric
)

# Add layers to the network
n_features = dataset.X.shape[1]

nn.add(DenseLayer(n_units=n_features // 2, input_shape=(n_features,)))  # Dense Layer 1 (Reduce units to half)
nn.add(ReLUActivation())                                                # ReLU activation layer

nn.add(DenseLayer(n_units=n_features // 2, input_shape=(n_features,)))  # Dense Layer 2 (Reduce units to half)
nn.add(ReLUActivation())                                                # ReLU activation layer

nn.add(DenseLayer(n_units=1, input_shape=(n_features,)))                # Output Dense Layer (Binary classification requires 1 unit)
nn.add(SigmoidActivation())                                             # Sigmoid activation layer

In [6]:
# Step 3: Perform k-fold cross validation
scores=k_fold_cross_validation(
    model=nn, 
    dataset=dataset, 
    scoring=accuracy, 
    cv=5, # 5-fold cross validation
    seed=42 
)

Epoch 1/100 - loss: 282.3491 - accuracy: 0.4825
Epoch 2/100 - loss: 279.9439 - accuracy: 0.5100
Epoch 3/100 - loss: 278.0281 - accuracy: 0.5175
Epoch 4/100 - loss: 277.6985 - accuracy: 0.5100
Epoch 5/100 - loss: 273.3217 - accuracy: 0.5675
Epoch 6/100 - loss: 273.4508 - accuracy: 0.5400
Epoch 7/100 - loss: 270.8651 - accuracy: 0.5575
Epoch 8/100 - loss: 270.9511 - accuracy: 0.5775
Epoch 9/100 - loss: 266.8565 - accuracy: 0.6075
Epoch 10/100 - loss: 267.4623 - accuracy: 0.6100
Epoch 11/100 - loss: 264.4678 - accuracy: 0.5775
Epoch 12/100 - loss: 267.1161 - accuracy: 0.5975
Epoch 13/100 - loss: 263.6767 - accuracy: 0.6175
Epoch 14/100 - loss: 262.2662 - accuracy: 0.6050
Epoch 15/100 - loss: 260.3648 - accuracy: 0.6050
Epoch 16/100 - loss: 261.1178 - accuracy: 0.6100
Epoch 17/100 - loss: 255.6599 - accuracy: 0.6050
Epoch 18/100 - loss: 260.8819 - accuracy: 0.6025
Epoch 19/100 - loss: 250.7425 - accuracy: 0.6475
Epoch 20/100 - loss: 256.8221 - accuracy: 0.6325
Epoch 21/100 - loss: 252.9485

In [7]:
# Step 4: Print Cross-Validation Results
print(f"K-Fold Cross Validation Scores: {scores}")
print(f"Mean Accuracy: {np.mean(scores):.2f}")

K-Fold Cross Validation Scores: [np.float64(0.53), np.float64(0.73), np.float64(0.7), np.float64(0.84), np.float64(0.93)]
Mean Accuracy: 0.75
